In [1]:
from fastai.vision.all import load_learner
import pandas as pd

In [2]:
learn = load_learner('models/convnext_base.pkl')

In [3]:
df1 = pd.read_csv('Groundtruths/MMAC2023_Spherical_Equivalent_Prediction_Training_Labels.csv')

In [6]:
df1.loc[:,'image'] = df1.image.apply(lambda x: 'Images/TrainingSet/'+x)

In [12]:
df2 = pd.read_csv('../1. Classification of Myopic Maculopathy/Groundtruths/MMAC2023_Myopic_Maculopathy_Classification_Training_Labels.csv')

In [13]:
df2.loc[:,'image'] = df2.image.apply(lambda x: '../1. Classification of Myopic Maculopathy/Images/TrainingSet/'+x)

In [21]:
df1['Split']=False

In [22]:
df2['Split']=True

In [16]:
df2['spherical_equivalent']=None

In [23]:
pd.concat([df1[['image','spherical_equivalent','Split']],df2[['image','spherical_equivalent','Split']]])

,image,spherical_equivalent,Split
0,Images/TrainingSet/mmac_task_3_train_0001.png,0.125,False
1,Images/TrainingSet/mmac_task_3_train_0002.png,0.000,False
2,Images/TrainingSet/mmac_task_3_train_0003.png,-0.125,False
3,Images/TrainingSet/mmac_task_3_train_0004.png,-0.750,False
4,Images/TrainingSet/mmac_task_3_train_0005.png,-0.125,False
...,...,...,...
1138,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_1139.png,NaN,True
1139,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_1140.png,NaN,True
1140,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_1141.png,NaN,True
1141,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_1142.png,NaN,True


In [24]:
from imutils import paths

In [30]:
res = []
for x in paths.list_images('../2. Segmentation of Myopic Maculopathy Plus Lesions/1. Lacquer Cracks/1. Images/1. Training Set/'):
    res.append([x,None,True])
for x in paths.list_images('../2. Segmentation of Myopic Maculopathy Plus Lesions/2. Choroidal Neovascularization/1. Images/1. Training Set/'):
    res.append([x,None,True])
for x in paths.list_images('../2. Segmentation of Myopic Maculopathy Plus Lesions/3. Fuchs Spot/1. Images/1. Training Set/'):
    res.append([x,None,True])

In [34]:
df3 = pd.DataFrame(res,columns=['image','spherical_equivalent','Split'])

In [36]:
dfNew = pd.concat([df1[['image','spherical_equivalent','Split']],df2[['image','spherical_equivalent','Split']],df3])

In [37]:
from fastai.vision.all import *


In [38]:
db = DataBlock(blocks = (ImageBlock, RegressionBlock),
             splitter=ColSplitter('Split'),
             get_x = ColReader(0),
             get_y=ColReader(1),
             item_tfms = [Resize(640)], # CropPad(200,200)
             batch_tfms=[*aug_transforms(size=512, min_scale=0.75,do_flip=True,flip_vert=True,
                  max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                         Normalize.from_stats(*imagenet_stats)])

In [39]:
dls = db.dataloaders(dfNew,bs=32)

In [40]:
learn.dls = dls

In [42]:
preds,y = learn.tta()

array(['../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_0001.png',
       '../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_0002.png',
       '../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_0003.png',
       ...,
       '../2. Segmentation of Myopic Maculopathy Plus Lesions/3. Fuchs Spot/1. Images/1. Training Set/mmac_task_2_train_FS_0016.png',
       '../2. Segmentation of Myopic Maculopathy Plus Lesions/3. Fuchs Spot/1. Images/1. Training Set/mmac_task_2_train_FS_0012.png',
       '../2. Segmentation of Myopic Maculopathy Plus Lesions/3. Fuchs Spot/1. Images/1. Training Set/mmac_task_2_train_FS_0036.png'],
      dtype=object)

In [50]:
res = []
for x,y in zip(dfNew[dfNew.Split==True]['image'],[x[0] for x in np.array(preds)]):
    res.append([x,y,False])

In [52]:
df4 = pd.DataFrame(res,columns=['image','spherical_equivalent','Split'])
df4

,image,spherical_equivalent,Split
0,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_0001.png,-3.241192,False
1,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_0002.png,-2.747184,False
2,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_0003.png,-2.974533,False
3,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_0004.png,-2.141968,False
4,../1. Classification of Myopic Maculopathy/Images/TrainingSet/mmac_task_1_train_0005.png,-6.224926,False
...,...,...,...
1289,../2. Segmentation of Myopic Maculopathy Plus Lesions/3. Fuchs Spot/1. Images/1. Training Set/mmac_task_2_train_FS_0009.png,-2.193742,False
1290,../2. Segmentation of Myopic Maculopathy Plus Lesions/3. Fuchs Spot/1. Images/1. Training Set/mmac_task_2_train_FS_0019.png,-11.204075,False
1291,../2. Segmentation of Myopic Maculopathy Plus Lesions/3. Fuchs Spot/1. Images/1. Training Set/mmac_task_2_train_FS_0016.png,-10.237436,False
1292,../2. Segmentation of Myopic Maculopathy Plus Lesions/3. Fuchs Spot/1. Images/1. Training Set/mmac_task_2_train_FS_0012.png,-2.716155,False


In [53]:
df1[['image','spherical_equivalent']]

,image,spherical_equivalent
0,Images/TrainingSet/mmac_task_3_train_0001.png,0.125
1,Images/TrainingSet/mmac_task_3_train_0002.png,0.000
2,Images/TrainingSet/mmac_task_3_train_0003.png,-0.125
3,Images/TrainingSet/mmac_task_3_train_0004.png,-0.750
4,Images/TrainingSet/mmac_task_3_train_0005.png,-0.125
...,...,...
987,Images/TrainingSet/mmac_task_3_train_0988.png,1.000
988,Images/TrainingSet/mmac_task_3_train_0989.png,-0.125
989,Images/TrainingSet/mmac_task_3_train_0990.png,-0.625
990,Images/TrainingSet/mmac_task_3_train_0991.png,-0.250


In [54]:
from sklearn.model_selection import train_test_split


In [58]:
train, test = train_test_split(df1[['image','spherical_equivalent']], test_size=0.2,random_state=5)

In [59]:
train['Split']=False
test['Split']=True

In [60]:
dfNew = pd.concat([df4,train,test])

In [63]:
dfNew.to_csv('dfNew.csv',index=None)